In [6]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import progressbar
from time import sleep
from datetime import datetime
import os

movie_dict = {'bitcoin': ['bitcoin since:2022-06-01 until:2022-06-17', 50000]}

In [7]:
today = datetime.today().strftime('%Y%m%d')[2:]+'_'
for index, movie_name in enumerate(movie_dict):
    print(movie_name, '%')
    tweets_list1 = []
    bar = progressbar.ProgressBar(maxval=movie_dict[movie_name][1]+2, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{movie_dict[movie_name][0]}').get_items()): #declare a username
        bar.update(i+1)
        if i>movie_dict[movie_name][1]: #number of tweets you want to scrape
            break
        #print(movie_name, i, tweet)
        tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username]) #declare the attributes to be returned
    tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

    tweets_df1[['Datetime', 'Text']].to_csv(f'{index}_BTC.csv')
    bar.finish()

bitcoin %


[=========================================================================] 100%


In [9]:
import pandas as pd

#when importing empty rows, they are transformed to nan, so we need to drop them here
df = pd.read_csv('0_BTC.csv', lineterminator='\n')[['Text']]
df

,Text
0,Dally found #bitcoin in a Blue vault at this l...
1,@Paul04Trader Bitcoin is that change. Only POW...
2,Bitcoin-boosting Salvadoran leader may have se...
3,#Bitcoin Last Price $21045 #BTC 🚀\nDaily Indic...
4,Y pensé que estas gorras ya se habían quedado ...
...,...
49996,@SativusRaphanus @Tyrexito Que invierte en Bit...
49997,@I3puttoften Only been day 2 that’s not a avg...
49998,"Bitcoin Price and Value Completely Separate, M..."
49999,The Giant Whale Speaks: Das ist die Einstiegss...


In [10]:
df.columns

Index(['Text'], dtype='object')

In [11]:
#get rid of links and hashtags
df["text"] = df["Text"].apply(lambda x : ' '.join([s for s in x.split(' ') if s.find('@') == -1 and s.find('www') == -1 and s.find('https') == -1]))

#get rid of non-ascii characters
df = df.replace(r'\W+', ' ', regex=True)
df

,Text,text
0,Dally found bitcoin in a Blue vault at this lo...,Dally found bitcoin in a Blue vault at this lo...
1,Paul04Trader Bitcoin is that change Only POW ...,Bitcoin is that change Only POW money Everythi...
2,Bitcoin boosting Salvadoran leader may have se...,Bitcoin boosting Salvadoran leader may have se...
3,Bitcoin Last Price 21045 BTC Daily Indicators...,Bitcoin Last Price 21045 BTC Daily Indicators...
4,Y pensé que estas gorras ya se habían quedado ...,Y pensé que estas gorras ya se habían quedado ...
...,...,...
49996,SativusRaphanus Tyrexito Que invierte en Bitc...,Que invierte en Bitcoin
49997,I3puttoften Only been day 2 that s not a avg ...,Only been day 2 that s not a avg price still A...
49998,Bitcoin Price and Value Completely Separate Mo...,Bitcoin Price and Value Completely Separate Mo...
49999,The Giant Whale Speaks Das ist die Einstiegsst...,The Giant Whale Speaks Das ist die Einstiegsst...


In [12]:
# python3 -m spacy download en_core_web_sm

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("spacytextblob")

df['sentiment'] = df['text'].apply(lambda x : nlp(x)._.polarity)
df_sentiment = df.sort_values('sentiment').reset_index(drop=True)
df_sentiment

,Text,text,sentiment
0,NYCMayor Way to go Mayor Adams The mining mor...,Way to go Mayor Adams The mining moratorium i...,-1.0
1,saylor On the contrary Bitcoin is the archite...,On the contrary Bitcoin is the architect of Ch...,-1.0
2,FCMB is the worst thing that will ever happen ...,FCMB is the worst thing that will ever happen ...,-1.0
3,CsokaDan BitcoinMagazine saylor Gold is liter...,Gold is literally a disgrace to humans at thi...,-1.0
4,Insane Bitcoin Pump Incoming Will Ethereum 2 E...,Insane Bitcoin Pump Incoming Will Ethereum 2 E...,-1.0
...,...,...,...
49996,bitcoin_apex_rv awesome,awesome,1.0
49997,Chomeofficial Awesome lfg clipper_corp Crypto...,Awesome lfg ETH CHOME Bitcoin Crypto Airdrop BNB,1.0
49998,felipe_brc Bitcoin isn t a inflation hedge It...,Bitcoin isn t a inflation hedge It a basically...,1.0
49999,Bubs1989 found bitcoin in a User vault at this...,Bubs1989 found bitcoin in a User vault at this...,1.0


In [13]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

#adding a counter to check the progress of the algo while it runs
global counter
counter = 0
def preprocess(sentence, stemming=False, lemmatizing=False):
  global counter
  counter += 1
  if counter % 100 == 0:
    pass
    #print(counter)

  #clean as much as possible, but not apply strong editing to the text, yet
  sentence=str(sentence)
  tokenizer = RegexpTokenizer(r'\w+')

  sentence = sentence.lower()
  sentence=sentence.replace('{html}',"") 
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', sentence)
  rem_url=re.sub(r'http\S+', '',cleantext)
  rem_num = re.sub('[0-9]+', '', rem_url)
  tokens = tokenizer.tokenize(rem_num)
  
  filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
  
  if stemming == True and lemmatizing == False:
    stem_words=[stemmer.stem(w) for w in filtered_words]
    return " ".join(stem_words)

  if stemming == False and lemmatizing == True:
    lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    return " ".join(lemma_words)

  if stemming == True and lemmatizing == True:
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)
  
  #at the end of the algo we return filtered words
  return " ".join(filtered_words)

#preprocess the sentiment text
df_sentiment['text'] = df_sentiment['text'].apply(lambda x: preprocess(x, stemming=False, lemmatizing=True))
df_sentiment

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aeshwihdi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aeshwihdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Text,text,sentiment
0,NYCMayor Way to go Mayor Adams The mining mor...,way mayor adam mining moratorium insane disast...,-1.0
1,saylor On the contrary Bitcoin is the archite...,contrary bitcoin architect chaos object greed ...,-1.0
2,FCMB is the worst thing that will ever happen ...,fcmb worst thing ever happen guy make lose dea...,-1.0
3,CsokaDan BitcoinMagazine saylor Gold is liter...,gold literally disgrace human point shameful b...,-1.0
4,Insane Bitcoin Pump Incoming Will Ethereum 2 E...,insane bitcoin pump incoming ethereum ever finish,-1.0
...,...,...,...
49996,bitcoin_apex_rv awesome,awesome,1.0
49997,Chomeofficial Awesome lfg clipper_corp Crypto...,awesome lfg eth chome bitcoin crypto airdrop bnb,1.0
49998,felipe_brc Bitcoin isn t a inflation hedge It...,bitcoin inflation hedge basically tech stock g...,1.0
49999,Bubs1989 found bitcoin in a User vault at this...,bubs found bitcoin user vault location join pl...,1.0


In [14]:
df_neg = df_sentiment[df_sentiment['sentiment'] < 0]
df_pos = df_sentiment[df_sentiment['sentiment'] > 0]
print('The number of neg tweets is ', len(df_neg))
print('The number of pos tweets is ', len(df_pos))
print('            ---                ')
if len(df_pos) > len(df_neg):
    print('Positive Teweets')
else:
    print('Negative Teweets')

The number of neg tweets is  6653
The number of pos tweets is  14926
            ---                
Positive Teweets


In [28]:
positive_words = pd.DataFrame([dict(Counter(' '.join(df_pos['text'].values.tolist()).split(' ')))]).T.sort_values(0, ascending=False)[0:100].index

negative_words = pd.DataFrame([dict(Counter(' '.join(df_neg['text'].values.tolist()).split(' ')))]).T.sort_values(0, ascending=False)[0:100].index

NameError: name 'Counter' is not defined

In [4]:
48155*(20 - 0.98606)*23

21059174.456100002

In [5]:
20*10000*12/23

104347.82608695653

In [1]:
3+1 

4